In [1]:
import numpy as np
import logging
import sys
from bitarray import bitarray, frozenbitarray
from bitarray.util import ba2int, int2ba
import pandas as pd



BIT_ORDER="little"
logging.basicConfig(format='%(message)s', datefmt='%m/%d/%Y %I:%M:%S %p', level=logging.DEBUG )
logger = logging.getLogger()

np.set_printoptions(threshold=sys.maxsize)
sub_s = "₀₁₂₃₄₅₆₇₈₉"

Function to encode the binary variables

In [2]:
def bin_encoder(v, l):
    y = bitarray(l, endian=BIT_ORDER)
    y[v] = 1
    return ba2int(y)

Function to create П matrix

In [3]:
def create_perm(C: np.array):
    perm = C.argsort(kind='quick', axis=0)
    return perm


In [4]:
def calculate_degree(y):
    y_bin = int2ba(y, endian=BIT_ORDER)
    degree = y_bin.count()
    return degree

Function to create the coefficients using the П matrix

In [5]:
def create_coeffs_matrix(C: np.array, perm: np.array, **kwargs):
    sorted_c = np.take_along_axis(C, perm, axis=0)
    zs = np.zeros((sorted_c.shape[1],), dtype=sorted_c.dtype)

    coeffs_c = np.vstack((sorted_c, zs)) - np.vstack((zs, sorted_c))
    coeffs_c = coeffs_c[:-1]

    return coeffs_c


Function to create the Boolean variables 

In [6]:

def create_variable_matrix(C: np.array, perm: np.array):
    y = perm[:-1]
    y_maker = np.frompyfunc(bin_encoder, 2, 1)
    
    y = y_maker(y, perm.shape[0])
    y = y.cumsum(axis=0)

    return y


Function to calculate the term's degree

In [7]:
def calculate_degree(y):
    y_bin = int2ba(y, endian=BIT_ORDER)
    degree = y_bin.count()
    return degree

Function to reduce the polynomial

In [8]:
def reduce_pbp_pandas(coeffs: np.array, variables: np.array):
    zero_vars = np.zeros((1, variables.shape[1]), dtype=int)
    var_flat = np.vstack([zero_vars, variables]).ravel()

    df = pd.DataFrame()
    
    df["y"] = var_flat
    df["coeffs"] = coeffs.ravel()
        

    df = df.groupby(['y'], as_index=False).agg({'y': 'first', 'coeffs': 'sum' })
    
    zero_coeffs = df["coeffs"] == 0
    df = df.loc[~zero_coeffs]

    df["y_str"] = df["y"].apply(decode_var)

    df["degree"] = df["y"].apply(calculate_degree)
    df.sort_values(by=['degree'], inplace=True)
    blankIndex=[''] * len(df)
    df.index=blankIndex


    return df

In [9]:
# Function to show terms added together as a polynomial 
def to_string(row):
    return f'{row["coeffs"]}{row["y_str"]}'

Function to decode Boolean variables

In [10]:
def decode_var(y):
    bin_indices = int2ba(y, endian=BIT_ORDER)
    y_arr  = np.frombuffer(bin_indices.unpack(), dtype=bool)
    indices = np.nonzero(y_arr)[0]

    if indices.size == 0:
        return ""

    return "y" + "y".join([sub_s[i+1] for i in indices])

Driver function to create a whole pBp

In [11]:
def create_pbp(c: np.array):
    assert len(c.shape) == 2
    perm_c = create_perm(c)
    coeffs_c = create_coeffs_matrix(c, perm_c)
    
    y = create_variable_matrix(c, perm_c)
    pBp = reduce_pbp_pandas(coeffs_c, y)
    
    return pBp


Function to truncate a pBp by a given p value

In [12]:
def truncate_pBp(pBp, c, p):
    cutoff = c.shape[0] - p + 1
    truncated_pBp = pBp.loc[pBp['degree'] < cutoff]
    return truncated_pBp 

In [17]:
def trunc_driver(c, p_list):
    print()
    print()
    pBp = create_pbp(c)
    print("Result pBp")
    polynomial = " + ".join(pBp.apply(to_string, axis=1))
    print(polynomial)
    print("=" * 100)

    for p in p_list:
        truncated_pBp = truncate_pBp(pBp, c, p)
        polynomial = " + ".join(truncated_pBp.apply(to_string, axis=1))
        print(f"p = {p}")
        print(polynomial)
        print("=" * 100)

Examples 

In [18]:
c = np.array([
        [7, 8, 2, 10, 3],
        [4, 12, 1, 8, 4],
        [5, 3, 0, 6, 9],
        [9, 6, 7, 1, 5]
    ])

trunc_driver(c, [2, 3, 4])



Result pBp
11 + 1y₁ + 1y₂ + 4y₃ + 5y₄ + 1y₁y₂ + 3y₂y₃ + 4y₃y₄ + 7y₁y₂y₃ + 4y₁y₂y₄ + 4y₁y₃y₄ + 2y₂y₃y₄
p = 2
11 + 1y₁ + 1y₂ + 4y₃ + 5y₄ + 1y₁y₂ + 3y₂y₃ + 4y₃y₄
p = 3
11 + 1y₁ + 1y₂ + 4y₃ + 5y₄
p = 4
11


In [19]:
c = np.array([
        [7, 15, 10, 7, 10],
        [10, 17, 4, 11, 22],
        [16, 7, 6, 18, 14],
        [11, 7, 6, 12, 8]
    ])

trunc_driver(c, [2, 3, 4])



Result pBp
33 + 7y₁ + 2y₂ + 2y₄ + 2y₁y₂ + 4y₁y₄ + 8y₃y₄ + 11y₁y₂y₄ + 10y₁y₃y₄ + 4y₂y₃y₄
p = 2
33 + 7y₁ + 2y₂ + 2y₄ + 2y₁y₂ + 4y₁y₄ + 8y₃y₄
p = 3
33 + 7y₁ + 2y₂ + 2y₄
p = 4
33


In [20]:
c = np.array([
        [0, 4, 6, 6, 6, 0, 32, 6],
        [12, 0, 4, 9, 4, 0, 28, 5],
        [9, 2, 0, 3, 3, 0, 20, 3],
        [6, 3, 2, 0, 4, 0, 24, 4],
        [18, 4, 6, 12, 0, 0, 32, 6],
        [15, 4, 4, 9, 5, 0, 12, 1],
        [24, 7, 10, 18, 8, 0, 0, 3],
        [18, 5, 6, 12, 6, 0, 12, 0]
    ])

trunc_driver(c, [2, 3, 4, 5, 6, 7])



Result pBp
6y₁ + 2y₂ + 2y₃ + 3y₄ + 3y₅ + 1y₈ + 12y₇ + 1y₂y₃ + 3y₁y₄ + 5y₃y₄ + 1y₃y₅ + 2y₆y₈ + 6y₁y₃y₄ + 1y₂y₃y₄ + 8y₆y₇y₈ + 5y₃y₆y₇y₈ + 1y₂y₃y₄y₅ + 2y₂y₃y₄y₆ + 3y₁y₂y₃y₄ + 1y₂y₃y₄y₅y₆ + 6y₁y₂y₃y₄y₆ + 5y₃y₄y₆y₇y₈ + 1y₁y₂y₃y₄y₅y₆ + 5y₂y₃y₄y₆y₇y₈ + 20y₁y₂y₃y₄y₅y₆y₈
p = 2
6y₁ + 2y₂ + 2y₃ + 3y₄ + 3y₅ + 1y₈ + 12y₇ + 1y₂y₃ + 3y₁y₄ + 5y₃y₄ + 1y₃y₅ + 2y₆y₈ + 6y₁y₃y₄ + 1y₂y₃y₄ + 8y₆y₇y₈ + 5y₃y₆y₇y₈ + 1y₂y₃y₄y₅ + 2y₂y₃y₄y₆ + 3y₁y₂y₃y₄ + 1y₂y₃y₄y₅y₆ + 6y₁y₂y₃y₄y₆ + 5y₃y₄y₆y₇y₈ + 1y₁y₂y₃y₄y₅y₆ + 5y₂y₃y₄y₆y₇y₈
p = 3
6y₁ + 2y₂ + 2y₃ + 3y₄ + 3y₅ + 1y₈ + 12y₇ + 1y₂y₃ + 3y₁y₄ + 5y₃y₄ + 1y₃y₅ + 2y₆y₈ + 6y₁y₃y₄ + 1y₂y₃y₄ + 8y₆y₇y₈ + 5y₃y₆y₇y₈ + 1y₂y₃y₄y₅ + 2y₂y₃y₄y₆ + 3y₁y₂y₃y₄ + 1y₂y₃y₄y₅y₆ + 6y₁y₂y₃y₄y₆ + 5y₃y₄y₆y₇y₈
p = 4
6y₁ + 2y₂ + 2y₃ + 3y₄ + 3y₅ + 1y₈ + 12y₇ + 1y₂y₃ + 3y₁y₄ + 5y₃y₄ + 1y₃y₅ + 2y₆y₈ + 6y₁y₃y₄ + 1y₂y₃y₄ + 8y₆y₇y₈ + 5y₃y₆y₇y₈ + 1y₂y₃y₄y₅ + 2y₂y₃y₄y₆ + 3y₁y₂y₃y₄
p = 5
6y₁ + 2y₂ + 2y₃ + 3y₄ + 3y₅ + 1y₈ + 12y₇ + 1y₂y₃ + 3y₁y₄ + 5y₃y₄ + 1y₃y₅ + 2y₆y₈ + 6y₁y₃y₄ + 1y₂y₃y₄ + 8y₆y₇y₈
p = 6
6